# Fuel Prices in France
This dashboard gives insight into the fuel prices in France. The data utilized is retreived from the [French Open Data Platform](https://www.data.gouv.fr/fr/datasets/prix-des-carburants-en-france-flux-quotidien/).

In [1]:
%%capture
!pip install bokeh

# Download and store the raw data

## Historical Prices

In [2]:
import requests
import os
import shutil
import zipfile
import xml.etree.ElementTree as ET
from datetime import datetime
import pandas
import duckdb
import json


def get_data_from_xml(url):
    destination_folder = "./raw_data/gasoline_france"
    destination_zip = f"{destination_folder}/data.zip"
    result = []
    def clean():
        """Prepare download folder"""
        if os.path.exists(destination_folder):
            shutil.rmtree(destination_folder)
        os.makedirs(destination_folder)
    def download():
        """Download zip file"""
        response = requests.get(url)
        with open(destination_zip, 'wb') as file:
            file.write(response.content)
    def unzip():
        with zipfile.ZipFile(destination_zip, 'r') as zip_ref:
            zip_ref.extractall(destination_folder)
    def load_xml():
        """Parse xml file"""
        xml_file_name = [k for k in os.listdir(destination_folder) if k.endswith('.xml')][0]
        xml_file_path = f"{destination_folder}/{xml_file_name}"
        tree = ET.parse(xml_file_path)
        root = tree.getroot()
        for price_node in root.findall('.//prix'):
            id = price_node.get('id')
            name = price_node.get('nom')
            maj = price_node.get('maj')
            if not maj:
                continue
            maj = datetime.fromisoformat(maj)
            price = price_node.get('valeur')
            result.append(
                    {
                        "id": id,
                        "name": name,
                        "maj": maj,
                        "price": float(price)
                    }
                )
    clean()
    download()
    unzip()
    load_xml()
    clean()
    return result

In [3]:
def get_historical_data_df():
    last_year = datetime.now().year - 1
    last_year_data = get_data_from_xml(f"https://donnees.roulez-eco.fr/opendata/annee/{last_year}")
    current_year_data = get_data_from_xml("https://donnees.roulez-eco.fr/opendata/annee")
    historical_data_df = pandas.DataFrame(last_year_data + current_year_data)
    return historical_data_df


def populate_daily_table():
    historical_data_df = get_historical_data_df()
    duckdb.sql("""
        create table daily as
            with
                dedupe as (
                    select
                        id,
                        name,
                        DATE_TRUNC('day', maj) as date_maj,
                        avg(cast(price as float)) as price
                    from historical_data_df
                    group by
                        id,
                        name,
                        date_maj
                )
            select
                name,
                date_maj,
                avg(price) as price
            from dedupe
            group by
                name,
                date_maj
    """)

In [4]:
populate_daily_table()
duckdb.sql("select * from daily")

┌─────────┬────────────┬────────────────────┐
│  name   │  date_maj  │       price        │
│ varchar │    date    │       double       │
├─────────┼────────────┼────────────────────┤
│ SP98    │ 2023-12-27 │  1.874142635874717 │
│ SP95    │ 2023-01-02 │ 1.8185397788510071 │
│ SP95    │ 2023-01-26 │ 1.9486574634999627 │
│ SP95    │ 2023-02-07 │ 1.9179069452659065 │
│ SP95    │ 2023-04-21 │  1.937398838163984 │
│ SP95    │ 2023-04-27 │ 1.9183163104518768 │
│ SP95    │ 2023-09-04 │  1.961939654987434 │
│ SP95    │ 2023-11-20 │  1.825539475264854 │
│ SP98    │ 2023-05-04 │ 1.9488609687009166 │
│ SP98    │ 2023-07-24 │ 1.9202409235677893 │
│  ·      │     ·      │          ·         │
│  ·      │     ·      │          ·         │
│  ·      │     ·      │          ·         │
│ E85     │ 2024-07-15 │ 0.8597499877214432 │
│ E85     │ 2024-08-06 │  0.847778450282726 │
│ E85     │ 2024-03-08 │ 0.9057061954151385 │
│ E85     │ 2024-03-23 │ 0.8988940614764973 │
│ E85     │ 2024-04-06 │  0.900589

## Current Prices

In [5]:
def populate_instantaneous_flux_table():
    result = []
    url = "https://data.economie.gouv.fr/api/explore/v2.1/catalog/datasets/prix-des-carburants-en-france-flux-instantane-v2/exports/json"
    data = requests.get(url).json()
    for row in data:
        if not row.get('prix'):
            continue
        id = row['id']
        code_departement = row['code_departement']
        prices = json.loads(row['prix'])
        if isinstance(prices, dict):
            prices = [prices]
        for price in prices:
            result.append(
                {
                    "id": id,
                    "code_departement": code_departement,
                    "name": price['@nom'],
                    "maj": datetime.fromisoformat(price['@maj']),
                    "price": float(price['@valeur'])
                }
            )
    instantaneous_flux_df = pandas.DataFrame(result)
    duckdb.sql("""
        create table instantaneous_flux as
            with
                dedupe as (
                    select *
                    from (
                        select
                            instantaneous_flux_df.*,
                            row_number() over (
                                partition by id, name
                                order by maj desc
                            ) as rn
                        from instantaneous_flux_df
                    ) as base
                    where rn = 1
                )
            select
                id,
                code_departement,
                name,
                price,
                date_trunc('day', maj) as date_maj
            from dedupe
    """)

In [6]:
populate_instantaneous_flux_table()
duckdb.sql("select * from instantaneous_flux")

┌──────────┬──────────────────┬─────────┬────────┬────────────┐
│    id    │ code_departement │  name   │ price  │  date_maj  │
│  int64   │     varchar      │ varchar │ double │    date    │
├──────────┼──────────────────┼─────────┼────────┼────────────┤
│  1000001 │ 01               │ SP98    │  1.828 │ 2024-08-29 │
│  1000004 │ 01               │ SP95    │   1.83 │ 2024-08-24 │
│  1000008 │ 01               │ Gazole  │  1.611 │ 2024-08-29 │
│  1000012 │ 01               │ E85     │  0.839 │ 2024-08-30 │
│  1000013 │ 01               │ Gazole  │  1.605 │ 2024-08-30 │
│  1100006 │ 01               │ Gazole  │  1.599 │ 2024-08-29 │
│  1100007 │ 01               │ E10     │  1.804 │ 2024-08-30 │
│  1120005 │ 01               │ E85     │  0.798 │ 2024-08-30 │
│  1130003 │ 01               │ Gazole  │  1.591 │ 2024-08-30 │
│  1150001 │ 01               │ Gazole  │  1.633 │ 2024-08-28 │
│     ·    │ ·                │   ·     │    ·   │     ·      │
│     ·    │ ·                │   ·     

In [7]:
is_daily_up_to_today = duckdb.sql("""
    select
        max(date_maj) in (select date_maj from daily)
    from instantaneous_flux
""").fetchall()[0][0]

if not is_daily_up_to_today:
    duckdb.sql("""
        insert into daily
            select
                name,
                date_maj,
                avg(price) as price
            from instantaneous_flux
            where date_maj in (
                select
                    max(date_maj)
                from instantaneous_flux
            )
            group by name, date_maj
    """)

# Metrics

In [8]:
prices_kpi_cards = duckdb.sql("""
    with
        kpis_dates as (
            select
                name,
                max(date_maj) as current_date,
                max(date_maj) - interval 7 day as last_week_date,
                max(date_maj) - interval 1 month as last_month_date,
                max(date_maj) - interval 1 year as last_year_date
            from daily
            group by name
        )
    select
        daily.name,
        max(
            case
                when date_maj = current_date and daily.name = kpis_dates.name then price
                else null
            end
        ) as today,
        max(
            case
                when date_maj = last_week_date and daily.name = kpis_dates.name then price
                else null
            end
        ) as last_week,
        max(
            case
                when date_maj = last_month_date and daily.name = kpis_dates.name then price
                else null
            end
        ) as last_month,
        max(
            case
                when date_maj = last_year_date and daily.name = kpis_dates.name then price
                else null
            end
        ) as last_year
    from daily, kpis_dates
    group by
        daily.name
""").df().to_dict('records')

In [9]:
prices_kpi_cards

[{'name': 'E10',
  'today': 1.7367681281618905,
  'last_week': 1.7559991624974398,
  'last_month': 1.805627308540044,
  'last_year': 1.9456964943045303},
 {'name': 'E85',
  'today': 0.8246892808683874,
  'last_week': 0.8472732133327548,
  'last_month': 0.8518453821222833,
  'last_year': 1.0511502293906048},
 {'name': 'SP95',
  'today': 1.759142857142857,
  'last_week': 1.7832625703819256,
  'last_month': 1.8291890631107597,
  'last_year': 1.966848582224129},
 {'name': 'SP98',
  'today': 1.845249560632689,
  'last_week': 1.861652765791602,
  'last_month': 1.908755576291458,
  'last_year': 2.0078125979205756},
 {'name': 'GPLc',
  'today': 0.9724874551971324,
  'last_week': 1.0049036351662108,
  'last_month': 1.005987844749787,
  'last_year': 0.9968871033106226},
 {'name': 'Gazole',
  'today': 1.6382305805396526,
  'last_week': 1.645897252402249,
  'last_month': 1.7055102185856912,
  'last_year': 1.8914288351424904}]

## Define the html to display the metrics

In [10]:
from IPython.display import display, HTML
import jinja2

def get_price_card(gasoline_type):
    for k in prices_kpi_cards:
        if k['name'] == gasoline_type:
            prices = k
            break
    prices['today_str'] = "{:.3f}".format(prices['today'])
    for k in ['last_week', 'last_month', 'last_year']:
        if not prices.get(k):
            prices[f'{k}_diff'] = False
            continue
        diff = prices['today'] - prices[k]
        prices[f'{k}_diff'] = diff
        diff_str = "{:.1f}".format(diff*100)
        if diff > 0:
            diff_str = '+' + diff_str
        prices[f'{k}_diff_str'] = diff_str
        prices[f'{k}_diff_color'] = 'red' if diff > 0 else 'green'
    template_html = jinja2.Template("""
        <div style="display: flex; justify-content: center; align-items: center;">
            <div style="text-align: center; margin: auto;">
                <h1 style="margin: 0">{{ name }}</h1>
                <h2 style="margin: 10px">{{ today_str }} €/l</h2>
                {% if last_week_diff %}
                    <hr style="margin: 0">
                    <span style="margin: 0; color: {{ last_week_diff_color }};">{{ last_week_diff_str }}</span>
                    cts €
                    <div>over a week</div>
                {% endif %}
                {% if last_month_diff %}
                    <hr style="margin: 0">
                    <span style="margin: 0; color: {{ last_month_diff_color }};">{{ last_month_diff_str }}</span>
                    cts €
                    <div>over a month</div>
                {% endif %}
                {% if last_year_diff %}
                    <hr style="margin: 0">
                    <span style="margin: 0; color: {{ last_year_diff_color }};">{{ last_year_diff_str }}</span>
                    cts €
                    <div>over a year</div>
                {% endif %}
            </div>
        </div>
    """)
    html = template_html.render(**prices)
    display(HTML(html))

In [11]:
current_date = duckdb.sql("""
        select
            cast(max(date_maj) as date) as current_date
        from daily
    """).df()['current_date'][0]
current_date = current_date.strftime("%B %d, %Y")
display(HTML(f'<p style="display: flex; justify-content: center; align-items: center;">Average Prices for {current_date}<p>'))

In [12]:
get_price_card('Gazole')

In [13]:
get_price_card('SP95')

In [14]:
get_price_card('SP98')

In [15]:
get_price_card('E10')

In [16]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

# Prices Evolution

In [17]:
from bokeh.palettes import Spectral4
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

p = figure(x_axis_type="datetime", title="Historical Average Prices")
p.title.text_font_size = '20pt'
p.title.align = 'center'
for gasoline, color in zip(['Gazole', 'SP95', 'SP98'], Spectral4):
#for gasoline, color in zip(['Gazole', 'SP95-E10'], Spectral4):
    data_df = duckdb.sql(f"""
        select *
        from daily
        where
            name = '{gasoline}'
        order by date_maj
    """).df()
    p.line(data_df['date_maj'], data_df['price'], line_width=2, color=color, legend_label=gasoline, name=gasoline)

hover = HoverTool(
    tooltips=[
        ("Gasoline", "$name"),
        ("Date", "@x{%F}"),
        ("Price", "@y")
    ],
    formatters={
        '@x': 'datetime',  # use 'datetime' formatter for 'x' values
        '@y': 'printf',  # use 'printf' formatter for 'y' values
    }
)
p.add_tools(hover)

show(p)

# Average Price per Departement

In [ ]:
from bokeh.models import LogColorMapper
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure, show
from bokeh.sampledata.unemployment import data as unemployment
from bokeh.sampledata.us_counties import data as counties
import numpy

# Average price for past 7 days
departement_e10 = duckdb.sql("""
    select
        code_departement,
        avg(price) as price
    from instantaneous_flux
    where
        date_maj >= (
            select
                max(date_maj) - interval 7 day
            from instantaneous_flux
        )
        and name = 'SP95'
    group by code_departement
""").df().to_dict('records') 
departement_e10 = {k["code_departement"]: k["price"] for k in departement_e10}

# Download geojson
url = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
departements_geojson = requests.get(url).json()

# Map grid
palette = tuple(reversed(palette))

x = []
y = []
names = []
codes = []
prices = []

for departement in departements_geojson['features']:
    for coordinates in departement['geometry']['coordinates']:
        departement_code = departement['properties']['code']
        price = departement_e10.get(departement_code, numpy.nan)
        prices += [price]
        names += [departement['properties']['nom']]
        coordinates = coordinates if departement['geometry']['type'] == 'Polygon' else coordinates[0]
        x += [[k[0] for k in coordinates]]
        y += [[k[1] for k in coordinates]]


data_map=dict(
    x=x,
    y=y,
    departements=names,
    prices=prices,
)

color_mapper = LogColorMapper(palette=palette)

p = figure(
    title="Average SP95 Prices per Department",
    x_axis_location=None, y_axis_location=None,
    tooltips=[
        ("Departement", "@departements"), ("Average Price", "@prices"),
    ])
p.title.text_font_size = '20pt'
p.title.align = 'center'
p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"

p.patches('x', 'y', source=data_map,
          fill_color={'field': 'prices', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

show(p)

# Price Distribution

In [ ]:
from bokeh.models import ColumnDataSource, Whisker, Range1d
from bokeh.transform import factor_cmap
from bokeh.models import HoverTool, GlyphRenderer
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

last_day = duckdb.sql("""
    with
        last_day as (
            select *
            from instantaneous_flux
            where
                date_maj = (
                    select
                        max(date_maj)
                    from instantaneous_flux
                )
                and name in ('Gazole', 'SP95', 'SP98')
        ),
        quantiles as (
            select
                name,
                quantile_cont(price, 0.25) as q1,
                quantile_cont(price, 0.5) as q2,
                quantile_cont(price, 0.75) as q3
            from last_day
            group by name
        ),
        iqr as (
            select
                name,
                q3-1.5*(q3-q1) as lower_limit,
                q3+1.5*(q3-q1) as upper_limit
            from quantiles
        )
    select
        last_day.name,
        last_day.price,
        quantiles.q1,
        quantiles.q2,
        quantiles.q3,
        iqr.lower_limit,
        iqr.upper_limit
    from last_day
    inner join quantiles
        on last_day.name = quantiles.name
    inner join iqr
        on iqr.name = last_day.name
""")
last_day_df = last_day.df()
box_plot_df = duckdb.sql("""
    select
        name,
        q1,
        q2,
        q3,
        lower_limit,
        upper_limit
    from last_day
    group by 1,2,3,4,5,6
""").df()

prix_nom = ['Gazole', 'SP95', 'SP98']
box_plot_source = ColumnDataSource(box_plot_df)

p = figure(x_range=prix_nom, tools="", toolbar_location=None,
           title="Price Distribution Across France",
           y_axis_label="Price in €",
           y_range=Range1d(start=min(last_day_df['price'])*0.96, end=max(last_day_df['price'])*1.01))

# outlier range
whisker = Whisker(base="name", upper="upper_limit", lower="lower_limit", source=box_plot_source)
whisker.upper_head.size = whisker.lower_head.size = 20
p.add_layout(whisker)


p.xgrid.grid_line_color = None
p.axis.major_label_text_font_size="14px"
p.axis.axis_label_text_font_size="12px"
p.title.text_font_size = '20pt'
p.title.align = 'center'

cmap = factor_cmap("name", palette=Spectral4, factors=prix_nom)
top_box = p.vbar("name", 0.7, "q2", "q3", source=box_plot_source, color=cmap, line_color="black")
bottom_box = p.vbar("name", 0.7, "q1", "q2", source=box_plot_source, color=cmap, line_color="black")
box_hover = HoverTool(renderers=[top_box, bottom_box],
                         tooltips=[
                             ('Gazyle Type', '@name'),
                             ('q3', '@q3€'),
                             ('q2', '@q2€'),
                             ('q1', '@q1€')
                         ])
p.add_tools(box_hover)


# outliers
outliers = last_day_df[~last_day_df.price.between(last_day_df.lower_limit, last_day_df.upper_limit)]
outliers_source = ColumnDataSource(outliers)
outliers_glyph = p.scatter("name", "price", source=outliers_source, size=6, color="black", alpha=0.3)
outliers_hover = HoverTool(renderers=[outliers_glyph],
                           tooltips=[
                               ('Price', '@price€')
                           ])
p.add_tools(outliers_hover)

show(p)